<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Setting up the ModelOps Environment
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Setup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>In this notebook we walk through setting up the ModelOps environment. We will cover how to connect to Vantage, create the necesarry DDLs and how to import the necessary data</p>


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Configure the Environment</b>

In [ ]:
%pip install teradataml==17.20.0.6 teradatamodelops==7.0.3 pandas==1.1.5


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</b>

In [1]:
from teradataml import (
    create_context, 
    execute_sql,
)
import getpass
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)


host = input("Host:")
username = input("Username:")
password = getpass.getpass("Password:")
database = input("Database (defaults to user):")

if not database:
    database = username


engine = create_context(host=host, 
                        username=username, 
                        password=password, 
                        logmech="TDNEGO",
                        database=database)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Create Tables</b>

<p style = 'font-size:18px;font-family:Arial'><b>Create BYOM models table</b></p>

In [4]:
query = '''CREATE SET TABLE DEMO_USER.Aoa_Byom_Models 
     (
      model_version VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_type VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      project_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      deployed_at TIMESTAMP(6) DEFAULT CURRENT_TIMESTAMP(6),
      model BLOB(2097088000))
UNIQUE PRIMARY INDEX ( model_version );
'''

try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DEMO_USER.Aoa_Byom_Models;')
    execute_sql(query)

<p style = 'font-size:18px;font-family:Arial'><b>Create predictions table</b></p>

In [5]:
query = '''CREATE MULTISET TABLE DEMO_USER.Pima_Patient_Predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      PatientId BIGINT,
      HasDiabetes BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
'''
 
try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DEMO_USER.Pima_Patient_Predictions;')
    execute_sql(query)

<p style = 'font-size:18px;font-family:Arial'><b>Compute statistics</b></p>

<p style = 'font-size:16px;font-family:Arial'>Compute the statistics metadata for the continuous variables</p>

```bash
aoa feature compute-stats \
        -s <feature-db>.<feature-data> \
        -m <statistics-metadata-db>.<statistics-metadata-table> \
        -t continuous -c numtimesprg,plglcconc,bloodp,skinthick,twohourserins,bmi,dipedfunc,age
```

<p style = 'font-size:16px;font-family:Arial'>Compute the statistics metadata for the categorical variables</p>

```bash
aoa feature compute-stats \
        -s <feature-db>.<feature-data> \
        -m <statistics-metadata-db>.<statistics-metadata-table> \
        -t categorical -c hasdiabetes
```

<p style = 'font-size:18px;font-family:Arial'><b>Importing Data</b></p>

In [ ]:
from teradataml import copy_to_sql, DataFrame
from teradatasqlalchemy.types import *
import pandas as pd

df = pd.read_csv("data/pima_patient_features.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_features",     
            schema_name=database,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "NumTimesPrg": INTEGER, 
                "PlGlcConc": INTEGER,
                "BloodP": INTEGER,
                "SkinThick": INTEGER,
                "TwoHourSerIns": INTEGER,
                "BMI": FLOAT,
                "DiPedFunc": FLOAT,
                "Age": INTEGER
            })

df = pd.read_csv("data/pima_patient_diagnoses.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_diagnoses",     
            schema_name=database,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "HasDiabetes": INTEGER
            })

# we can compute this from the CLI also - but lets import pre-computed for now.
df = pd.read_csv("data/aoa_statistics_metadata.csv")
copy_to_sql(df=df, 
            table_name="aoa_statistics_metadata",     
            schema_name=database,
            if_exists="append")


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>ModelOps UI</b>

<p style = 'font-size:18px;font-family:Arial'><b>Model Lifecycle</b></p>

1. Python Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Check Deployments/executions
   - Retire

2. R Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Check Deployments/executions
   - Retire

3. BYOM Diabetes Prediction
   - Run BYOM Notebook 
   - Define BYOM Model 
   - Import Version
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Check Deployments/executions
   - Retire

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>View Predictions</b>

<p style = 'font-size:16px;font-family:Arial'>In the UI, select a deployment from the deployments left hand navigation. Go to the Jobs tab and on the right hand side for each job execution, you can select "View Predictions". This will show you a sample of the predictions for that particular job execution.</p>

<p style = 'font-size:16px;font-family:Arial'>Note, your predictions table must have a `job_id` column which matches to the execution job id. If using BYOM, this is done automatically. For you own `scoring.py`, checkout the demo models.</p>

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>CLI</b>


```bash
pip install aoa>=7.0.0rc3
```

<p style = 'font-size:18px;font-family:Arial'><b>Copy CLI Config</b></p>

```
Copy the CLI config from ModelOps UI -> Session Details -> CLI config
```

<p style = 'font-size:18px;font-family:Arial'><b>Add Dataset Connection</b></p>

```bash
aoa connection add
```

<p style = 'font-size:18px;font-family:Arial'><b>List Feature Metadata</b></p>

```bash
aoa feature list-stats -m {your-db}.aoa_feature_metadata
```

<p style = 'font-size:18px;font-family:Arial'><b>Clone Project</b></p>

```bash
aoa clone 
```

```bash
cd modelops-demo-models
```

<p style = 'font-size:18px;font-family:Arial'><b>Install Model Dependencies</b></p>

```bash
pip install -r model_definitions/python-diabetes/model_modules/requirements.txt
```

<p style = 'font-size:18px;font-family:Arial'><b>Train Model</b></p>

```bash
aoa run
```

<p style = 'font-size:18px;font-family:Arial'><b>Add model</b></p>

```bash
aoa add
```
